In [2]:
import pyspark



# ✅ PySpark Top 100 Methods — **PART 1 (1–25)**


**Category: DataFrame Creation, Selection, Filtering, Transformation**




## 1️⃣ `SparkSession.builder.getOrCreate()`

### **What**

Creates or retrieves the active Spark session.

### **Why**

Entry point for **all PySpark operations** (DataFrame, SQL, streaming).

### **How**

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.master("local[*]") \
.config("spark.app.name", "MyApp") \
.config("spark.driver.host", "127.0.0.1") \
.config("spark.driver.bindAddress", "127.0.0.1") \
.config("spark.driver.port", 4040) \
.config("spark.blockManager.port", 4041) \
.getOrCreate()


### **When / Scenario**

* First line in **every PySpark job**
* Required for DataFrame & SQL APIs

### **Interview Tip**

> SparkSession **replaced SparkContext + SQLContext + HiveContext**



## 2️⃣ `spark.read`

### **What**

Entry point to read external data.

### **Why**

Supports multiple formats (CSV, JSON, Parquet, ORC, Avro).

### **How**

In [2]:
import pathlib

In [3]:

filepath = str(pathlib.Path().cwd().parent / 'data' / 'Spotify_Artists.csv')
filepath

'D:\\shra1\\github\\pyspark-practice\\data\\Spotify_Artists.csv'

In [4]:
df = spark.read.csv(filepath,
                    header=True,
                    inferSchema=True)

In [5]:
df.show(5)

+---------+--------+----------+---------+
|artist_id|    name|     genre|  country|
+---------+--------+----------+---------+
|        1|Artist_1|Electronic|   France|
|        2|Artist_2|Electronic|Australia|
|        3|Artist_3|      Jazz|   France|
|        4|Artist_4| Classical|Australia|
|        5|Artist_5|   Hip-Hop|      USA|
+---------+--------+----------+---------+
only showing top 5 rows


### **Scenario**

* Data ingestion layer
* Batch pipelines




## 3️⃣ `spark.read.format()`

### **What**

Explicitly defines file format.

### **Why**

Needed for advanced formats (Parquet, Delta, JDBC).

### **How**

In [7]:
# df = spark.read.format("parquet").load("s3://bucket/data")

### **Interview Angle**

> Preferred over `.csv()` / `.json()` in **production pipelines**




## 4️⃣ `spark.read.option()`

### **What**

Sets read-time options.

### **Why**

Controls parsing behavior (delimiter, schema, encoding).

### **How**

In [ ]:
# df = spark.read.option("delimiter", "|").csv("data.txt")



## 5️⃣ `df.show()`

### **What**

Displays rows.

### **Why**

Quick debugging & inspection.

### **How**

In [6]:
df.show(5, truncate=False)

+---------+--------+----------+---------+
|artist_id|name    |genre     |country  |
+---------+--------+----------+---------+
|1        |Artist_1|Electronic|France   |
|2        |Artist_2|Electronic|Australia|
|3        |Artist_3|Jazz      |France   |
|4        |Artist_4|Classical |Australia|
|5        |Artist_5|Hip-Hop   |USA      |
+---------+--------+----------+---------+
only showing top 5 rows


### **Interview Trick**

> `show()` **triggers an action**




## 6️⃣ `df.printSchema()`

### **What**

Displays schema tree.

### **Why**

Critical for **debugging type issues**.

### **How**

In [7]:
df.printSchema()

root
 |-- artist_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- country: string (nullable = true)





## 7️⃣ `df.schema`

### **What**

Returns schema object.

### **Why**

Used for **programmatic schema validation**.

### **How**

In [8]:
df.schema

StructType([StructField('artist_id', IntegerType(), True), StructField('name', StringType(), True), StructField('genre', StringType(), True), StructField('country', StringType(), True)])



## 8️⃣ `df.select()`

### **What**

Selects columns.

### **Why**

Column pruning → performance optimization.

### **How**

In [9]:
df.select("name", "country").show(5)

+--------+---------+
|    name|  country|
+--------+---------+
|Artist_1|   France|
|Artist_2|Australia|
|Artist_3|   France|
|Artist_4|Australia|
|Artist_5|      USA|
+--------+---------+
only showing top 5 rows




## 9️⃣ `df.selectExpr()`

### **What**

Select using SQL expressions.

### **Why**

Cleaner transformations without `withColumn`.

### **How**

In [10]:
df.selectExpr("country as nation").show(5)

+---------+
|   nation|
+---------+
|   France|
|Australia|
|   France|
|Australia|
|      USA|
+---------+
only showing top 5 rows


### **Interview**

> Faster for **simple derived columns**




## 🔟 `df.withColumn()`

### **What**

Adds or replaces a column.

### **Why**

Core transformation API.

### **How**

In [12]:
from pyspark.sql.functions import col

df = df.withColumn("id_with_name", col("artist_id").cast("string") + "_" + col("name"))
df.show(5)

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

### **Interview Warning**

> Multiple `withColumn()` calls = **multiple DAG stages**

---

## 1️⃣1️⃣ `df.withColumnRenamed()`

### **What**

Renames a column.

### **How**

In [ ]:
df = df.withColumnRenamed("sal", "salary")

---

## 1️⃣2️⃣ `df.drop()`

### **What**

Removes columns.

### **Why**

Reduce memory & shuffle size.

### **How**

In [ ]:
df.drop("temp_col")

---

## 1️⃣3️⃣ `df.filter()` / `df.where()`

### **What**

Row-level filtering.

### **Why**

Predicate pushdown optimization.

### **How**

In [ ]:
df.filter(col("salary") > 50000)

### **Interview**

> `filter()` and `where()` are **identical**

---

## 1️⃣4️⃣ `df.distinct()`

### **What**

Removes duplicate rows.

### **Why**

Data deduplication.

### **How**

In [ ]:
df.distinct()

### **Cost**

⚠️ Triggers **shuffle**

---

## 1️⃣5️⃣ `df.dropDuplicates()`

### **What**

Removes duplicates based on columns.

### **How**

In [ ]:
df.dropDuplicates(["id"])

### **Why**

More control than `distinct()`

---

## 1️⃣6️⃣ `df.orderBy()` / `df.sort()`

### **What**

Sorts rows.

### **How**

In [ ]:
df.orderBy(col("salary").desc())

### **Interview**

> Sorting causes **wide transformation (shuffle)**

---

## 1️⃣7️⃣ `df.limit()`

### **What**

Restricts number of rows.

### **Why**

Sampling / debugging.

### **How**

In [ ]:
df.limit(10)

---

## 1️⃣8️⃣ `df.sample()`

### **What**

Random sampling.

### **How**

In [ ]:
df.sample(fraction=0.1, seed=42)

### **Scenario**

* Model training
* Data analysis

---

## 1️⃣9️⃣ `df.count()`

### **What**

Counts rows.

### **Why**

Validation & metrics.

### **Interview**

> Very expensive on large datasets

---

## 2️⃣0️⃣ `df.collect()`

### **What**

Brings data to driver.

### **Why**

Debugging small datasets.

### **Danger**

❌ **OOM risk**

---

## 2️⃣1️⃣ `df.take()`

### **What**

Returns first N rows.

### **How**

In [ ]:
df.take(5)

---

## 2️⃣2️⃣ `df.head()`

### **What**

Same as `take(1)`.

---

## 2️⃣3️⃣ `df.columns`

### **What**

List of column names.

### **How**

In [ ]:
df.columns

---

## 2️⃣4️⃣ `df.dtypes`

### **What**

Column names with data types.

---

## 2️⃣5️⃣ `df.describe()`

### **What**

Basic statistics.

### **How**

In [ ]:
df.describe("salary").show()

---

# 🔥 Interview Coverage Achieved (Part 1)

✔ Data ingestion
✔ Column operations
✔ Filtering
✔ Performance implications
✔ Lazy vs action operations

---

## 👉 Next Parts (coming next)

**PART 2:** Aggregations, GroupBy, Joins (26–50)
**PART 3:** Functions, Window, UDF, Date/JSON (51–75)
**PART 4:** Performance, Partitioning, Caching, Writing, Spark Internals (76–100)

If you want, I can:

* Map **each method → SQL equivalent**
* Add **real interview questions per method**
* Add **Spark execution plan & DAG explanations**

Just say **“Continue with Part 2”**